In [1]:
!pip install av datasets
!pip install transformers torch
!pip install rouge_score
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12=

In [2]:
import json
import multiprocessing
import os

import av
from datasets import load_dataset
import numpy as np
from transformers import AutoTokenizer, AutoImageProcessor

2025-04-20 11:29:54.486234: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745148594.692637      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745148594.758202      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Login to Huggingface

In [3]:
from huggingface_hub import notebook_login

notebook_login()  # will prompt you for your HF token

In [4]:
# import os

# # Load Hugging Face token from Kaggle secret
# hf_token = os.environ.get("HF_TOKEN")
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")

# Use it in your login
from huggingface_hub import login
login(token=secret_value_0)


In [5]:
# # prompt: convert csv to df

# import pandas as pd

# # Assuming your CSV file is named 'your_file.csv' and is in the current directory.
# # Replace 'your_file.csv' with the actual path to your CSV file if it's located elsewhere.
# df = pd.read_csv('/kaggle/input/splitted-ucf-120videospercategory/splitted-ucf-clips.csv')
# # Remove "./splitted_clips/" from the 'video_path' column
# df['video_path'] = df['video_path'].str.replace('./splitted_clips/', '')
# # Display the first few rows of the DataFrame to verify
# df
# import pandas as pd
# from sklearn.model_selection import GroupShuffleSplit
# from datasets import Dataset

# # Assuming 'df' is your DataFrame and it includes a 'video_path' column
# df['video_group'] = df['video_path'].str.split('_').str[0]  # Adjust based on actual naming

# # Step 1: Train-Test split (85% train+val, 15% test)
# gss1 = GroupShuffleSplit(test_size=0.05, n_splits=1, random_state=42)
# train_val_inds, test_inds = next(gss1.split(df, groups=df['video_group']))
# train_val_df = df.iloc[train_val_inds].reset_index(drop=True)
# test_df = df.iloc[test_inds].reset_index(drop=True)

# # Step 2: Train-Validation split (from 85% of original data → 70% train, 15% val overall)
# gss2 = GroupShuffleSplit(test_size=0.1, n_splits=1, random_state=42)  
# # 0.1765 ≈ 15 / 85 → ensures ~15% of full data goes to validation
# train_inds, val_inds = next(gss2.split(train_val_df, groups=train_val_df['video_group']))
# train_df = train_val_df.iloc[train_inds].reset_index(drop=True)
# valid_df = train_val_df.iloc[val_inds].reset_index(drop=True)

# # Shuffle for randomness
# train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
# valid_df = valid_df.sample(frac=1, random_state=42).reset_index(drop=True)
# test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

# # Convert to Hugging Face Datasets format
# dataset = {
#     "train": Dataset.from_pandas(train_df[:50]),
#     "validation": Dataset.from_pandas(valid_df[:10]),
#     "test": Dataset.from_pandas(test_df[:10])
# }

# # Display sizes
# print(f"Train: {len(train_df)}, Validation: {len(valid_df)}, Test: {len(test_df)}")



In [6]:
# from datasets import DatasetDict
# image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# tokenizer.pad_token = tokenizer.eos_token

# # create dataset with features: videoID, pixel_values (8, 3, 224, 224), labels (10, 1024)
# def process(example):
#     videoID, captions = example["video_path"], example["caption"]
    
#     videos_path = "/kaggle/input/splitted-ucf-120videospercategory/splitted-ucf-clips"
#     video_path = os.path.join(videos_path, videoID)
#     if not os.path.isfile(video_path):
#         video_path = os.path.join(videos_path, videoID)
#     container = av.open(video_path)
    
#     # discrepancy between in codec metadata, manually get frame count
#     container.seek(0)
#     frame_count = 0
#     for frame in container.decode(video=0):
#         frame_count += 1
    
#     indices = set(np.linspace(0, frame_count, num=8, endpoint=False).astype(np.int64))
#     frames = []
#     container.seek(0)
#     for i, frame in enumerate(container.decode(video=0)):
#         if i in indices:
#             frames.append(frame.to_ndarray(format="rgb24"))   
            
#     pixel_values = image_processor(frames).pixel_values[0]
#     labels = tokenizer(captions, padding="max_length").input_ids
#     return {"videoID": videoID, "pixel_values": pixel_values, "labels": labels}

# dataset["train"] = dataset["train"].map(
#     process,
#     remove_columns=["caption"],
#     num_proc=1,
#     cache_file_name="/kaggle/tmp/dataset-ucf/cache",    # ← write here
#     writer_batch_size=500                          # flush every 500 examples
# )
# dataset["validation"] = dataset["validation"].map(process, remove_columns=["caption"], num_proc=1)
# dataset["test"] = dataset["test"].map(process, remove_columns=["caption"], num_proc=1)
# from datasets import DatasetDict

# # Wrap your dict into a DatasetDict
# dataset_dict = DatasetDict({
#     "train": dataset["train"],
#     "validation": dataset["validation"],
#     "test": dataset["test"]
    
# })
# from datasets import DatasetDict
# dataset = DatasetDict(dataset)
# #dataset.save_to_disk("./dataset-ucf", num_proc=1)

## Load Mapped Data from Huggingface

In [7]:
from datasets import load_dataset

# 1. Load all splits at once
dataset = load_dataset("NourFakih/ucf101-captioned-mapped")  
# `dataset` is a DatasetDict with keys like "train", "validation", "test" :contentReference[oaicite:0]{index=0}

train_ds      = dataset["train"]
validation_ds = dataset["validation"]
test_ds       = dataset["test"]

README.md:   0%|          | 0.00/723 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/82 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/82 [00:00<?, ?it/s]

train-00000-of-00082.parquet:   0%|          | 0.00/125M [00:00<?, ?B/s]

train-00001-of-00082.parquet:   0%|          | 0.00/130M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8397 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/82 [00:00<?, ?it/s]

## Load Base model

In [8]:
import json
import os

from datasets import Dataset, load_from_disk
import numpy as np
import torch
from torch.cuda.amp import GradScaler
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
from transformers import VisionEncoderDecoderModel, AutoImageProcessor, AutoTokenizer, default_data_collator, get_scheduler

device = "cuda"

# MODEL
encoder = "facebook/timesformer-base-finetuned-k600"
decoder = "gpt2"

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
tokenizer = AutoTokenizer.from_pretrained(decoder)
tokenizer.pad_token = tokenizer.eos_token

model_name="Neleac/SpaceTimeGPT"
model = VisionEncoderDecoderModel.from_pretrained(model_name).to(device)
#model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encoder, decoder).to(device)
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.max_length = 50
model.config.num_beams = 4
model.config.early_stopping = True


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/41.2k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.timesformer.modeling_timesformer.TimesformerModel'> is overwritten by shared encoder config: TimesformerConfig {
  "architectures": [
    "TimesformerForVideoClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "attention_type": "divided_space_time",
  "drop_path_rate": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "abseiling",
    "1": "acting in play",
    "2": "adjusting glasses",
    "3": "air drumming",
    "4": "alligator wrestling",
    "5": "answering questions",
    "6": "applauding",
    "7": "applying cream",
    "8": "archaeological excavation",
    "9": "archery",
    "10": "arguing",
    "11": "arm wrestling",
    "12": "arranging flowers",
    "13": "assembling bicycle",
    "14": "assembling computer",
    "15": "attending conference",
    "16": "auctioning",
    "17": "backflip (human)",
    "18": "baking cookies",
    "19": "bandaging",
    "20": "barb

## Prepare Data for training

In [9]:
# ───────────────────────────────────────────────────────────────────────────────
# Disable tokenizers parallelism to avoid deadlocks
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Imports
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import VisionEncoderDecoderModel , get_scheduler
from transformers import default_data_collator
from torch.amp import autocast, GradScaler
from datasets import load_from_disk

# ───────────────────────────────────────────────────────────────────────────────

In [10]:
# ───────────────────────────────────────────────────────────────────────────────
# Disable tokenizers parallelism to avoid deadlocks
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Imports
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import VisionEncoderDecoderModel , get_scheduler
from transformers import default_data_collator
from torch.amp import autocast, GradScaler
from datasets import load_from_disk

# # ───────────────────────────────────────────────────────────────────────────────
# 1. Dataset definition and loading
class UCFDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
         example = self.dataset[idx]
         label = example["labels"]
         if isinstance(label, list):
                  label = label[0]
         return {
            "video_path":   example["video_path"],
            "pixel_values": example["pixel_values"],
            "labels":  label     
         }
     


# load from disk and set PyTorch format
#dataset = load_from_disk("./dataset-ucf")
dataset.set_format("torch")
dataset_train = UCFDataset(dataset["train"])
dataset_val   = UCFDataset(dataset["validation"])

# ───────────────────────────────────────────────────────────────────────────────
# 2. Collators to ensure correct tensor shapes
def train_collator(examples):
    pixel_values = torch.stack([ex["pixel_values"] for ex in examples])
    labels       = torch.stack([ex["labels"]       for ex in examples])
    return {"pixel_values": pixel_values, "labels": labels}

def val_collator(examples):
    pixel_values = torch.stack([ex["pixel_values"] for ex in examples])
    labels       = torch.stack([ex["labels"]       for ex in examples])
    video_paths  = [ex["video_path"] for ex in examples]
    return {"pixel_values": pixel_values, "labels": labels, "video_paths": video_paths}

# ───────────────────────────────────────────────────────────────────────────────
# 3. DataLoaders
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataloader = DataLoader(
    dataset_train,
    collate_fn = train_collator,
    batch_size  = 1,
    drop_last   = True,
    num_workers = 1,
    pin_memory  = True,
)
val_dataloader = DataLoader(
    dataset_val,
    collate_fn = val_collator,
    batch_size  = 1,
    drop_last   = False,
    num_workers = 1,
    pin_memory  = True,
)


In [11]:

# ───────────────────────────────────────────────────────────────────────────────
# 4. Model, optimizer, scheduler, and AMP scaler
model     = VisionEncoderDecoderModel.from_pretrained("Neleac/SpaceTimeGPT").to(device)
#optimizer = AdamW(model.parameters(), lr=5e-7)
optimizer = AdamW(model.parameters(), lr=1e-6)
epochs    = 4
total_steps = epochs * len(train_dataloader)
scheduler   = get_scheduler(
    name             = "linear",
    optimizer        = optimizer,
    num_warmup_steps = 0,
    num_training_steps = total_steps,
)
scaler = GradScaler("cuda")  # :contentReference[oaicite:0]{index=0}

Config of the encoder: <class 'transformers.models.timesformer.modeling_timesformer.TimesformerModel'> is overwritten by shared encoder config: TimesformerConfig {
  "architectures": [
    "TimesformerForVideoClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "attention_type": "divided_space_time",
  "drop_path_rate": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "abseiling",
    "1": "acting in play",
    "2": "adjusting glasses",
    "3": "air drumming",
    "4": "alligator wrestling",
    "5": "answering questions",
    "6": "applauding",
    "7": "applying cream",
    "8": "archaeological excavation",
    "9": "archery",
    "10": "arguing",
    "11": "arm wrestling",
    "12": "arranging flowers",
    "13": "assembling bicycle",
    "14": "assembling computer",
    "15": "attending conference",
    "16": "auctioning",
    "17": "backflip (human)",
    "18": "baking cookies",
    "19": "bandaging",
    "20": "barb

####  Video Captioning Training

In [12]:
# from tqdm.auto import tqdm  # auto handles Jupyter vs script

# # Training loop with visible tqdm
# for epoch in range(epochs):
#     model.train()
#     train_loss = 0.0
#     loop = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs} - Training", leave=True)
    
#     for batch in loop:
#         pixel_values = batch["pixel_values"].to(device)
#         labels = batch["labels"]
#         if labels.dim() == 1:
#             labels = labels.unsqueeze(0)
#         labels = labels.to(device)

#         optimizer.zero_grad()
#         with autocast(device_type="cuda", dtype=torch.float16):
#             outputs = model(pixel_values=pixel_values, labels=labels)
#             loss = outputs.loss
#             train_loss += loss.item()

#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()
#         scheduler.step()

#         loop.set_postfix(loss=loss.item())  # update progress bar with current loss

#     # clear unused GPU memory to reduce fragmentation :contentReference[oaicite:2]{index=2}
#     torch.cuda.empty_cache()

#     # --- Validation ---
#     model.eval()
#     val_loss = 0.0
#     with torch.no_grad():
#         for batch in val_dataloader:
#             pixel_values = batch["pixel_values"].to(device)
#             labels       = batch["labels"]
#             if labels.dim() == 1:
#                 labels = labels.unsqueeze(0)
#             labels = labels.to(device)

#             with autocast(device_type="cuda", dtype=torch.float16):
#                 outputs = model(pixel_values=pixel_values, labels=labels)
#                 val_loss += outputs.loss.item()

#     # logging
#     avg_train = train_loss / len(train_dataloader)
#     avg_val   = val_loss   / len(val_dataloader)
#     print(f"Epoch {epoch+1} — train_loss: {avg_train:.4f}, val_loss: {avg_val:.4f}")

#     # save checkpoint
#     model.save_pretrained(f"./training-ucf/checkpoint_{epoch+1}")
# # ───────────────────────────────────────────────────────────────────────────────

##  TimeSformer+GPT2 Training and Validation with Rouge Metric

In [13]:
# !pip install rouge_score
# !pip install evaluate

In [14]:
# import os
# import shutil
# import torch
# import numpy as np
# import nltk
# from tqdm.auto import tqdm
# from huggingface_hub import Repository
# import evaluate

# # ─── Setup Rouge ───────────────────────────────────────────────────────────────
# nltk.download("punkt")
# rouge = evaluate.load("rouge")  # Load ROUGE metric :contentReference[oaicite:0]{index=0}

# def postprocess_text(preds, labels):
#     preds = [p.strip() for p in preds]
#     labels = [l.strip() for l in labels]
#     preds = ["\n".join(nltk.sent_tokenize(p)) for p in preds]
#     labels = ["\n".join(nltk.sent_tokenize(l)) for l in labels]
#     return preds, labels

# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
#     decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
#     result = rouge.compute(predictions=decoded_preds,
#                            references=decoded_labels,
#                            use_stemmer=True)
#     # scale to percentages
#     return {k: round(v * 100, 4) for k, v in result.items()}


In [15]:


# ─── Prepare HF Hub Repo ────────────────────────────────────────────────────
# Clone (or create) a HF repo into "./training-ucf"
# repo = Repository(
#     local_dir="training-ucf",
#     clone_from="NourFakih/TimeSformer-GPT2-UCF-7000",
#     use_auth_token=True
# )  # uses git under the hood :contentReference[oaicite:1]{index=1}

from huggingface_hub import HfApi, Repository  # install via pip install huggingface_hub

# 1) Create an empty model repo on HF (won’t error if it already exists)
api = HfApi()
api.create_repo(
    repo_id="NourFakih/TimeSformer-GPT2-UCF-8400",
    repo_type="model",
    exist_ok=True
)  # :contentReference[oaicite:0]{index=0}

# 2) Now clone that empty repo into your local folder
repo = Repository(
    local_dir="training-ucf",                          # where on disk to put it
    clone_from="NourFakih/TimeSformer-GPT2-UCF-8400",   # the HF namespace/repo
    use_auth_token=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/NourFakih/TimeSformer-GPT2-UCF-8400 into local empty directory.


In [16]:
# # ─── Training Loop ─────────────────────────────────────────────────────────────
# for epoch in range(epochs):
#     # — Training —
#     model.train()
#     train_loss = 0.0
#     pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs} • Training", leave=True)
#     for batch in pbar:
#         pixel_values = batch["pixel_values"].to(device)
#         labels = batch["labels"]
#         if labels.dim() == 1: labels = labels.unsqueeze(0)
#         labels = labels.to(device)

#         optimizer.zero_grad()
#         with autocast(device_type="cuda", dtype=torch.float16):
#             outputs = model(pixel_values=pixel_values, labels=labels)
#             loss = outputs.loss
#             train_loss += loss.item()
#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()
#         scheduler.step()

#         pbar.set_postfix(loss=loss.item())

#     torch.cuda.empty_cache()

#     # — Validation & ROUGE —
#     model.eval()
#     val_loss = 0.0
#     all_preds, all_labels = [], []
#     with torch.no_grad():
#         for batch in val_dataloader:
#             pixel_values = batch["pixel_values"].to(device)
#             labels = batch["labels"]
#             if labels.dim() == 1: labels = labels.unsqueeze(0)
#             labels = labels.to(device)

#             # 1) compute loss
#             with autocast(device_type="cuda", dtype=torch.float16):
#                 outputs = model(pixel_values=pixel_values, labels=labels)
#                 val_loss += outputs.loss.item()

#             # 2) generate predictions for ROUGE
#             generated_ids = model.generate(
#                 pixel_values,
#                 max_length=labels.shape[-1],       # adjust as needed
#                 num_beams=4,                       # optional
#                 early_stopping=True
#             )
#             all_preds.extend(generated_ids.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())

#     avg_train = train_loss / len(train_dataloader)
#     avg_val   = val_loss   / len(val_dataloader)
#     rouge_scores = compute_metrics((np.array(all_preds), np.array(all_labels)))
#     print(f"Epoch {epoch+1} — train_loss: {avg_train:.4f}, "
#           f"val_loss: {avg_val:.4f}, ROUGE: {rouge_scores}")

#     # — Keep only latest checkpoint locally —
#     prev_ckpt = f"./training-ucf/checkpoint_{epoch}"
#     if epoch > 0 and os.path.isdir(prev_ckpt):
#         shutil.rmtree(prev_ckpt)  # recursively delete old dir :contentReference[oaicite:2]{index=2}

#     # — Save new checkpoint —
#     ckpt_dir = f"./training-ucf/checkpoint_{epoch+1}"
#     model.save_pretrained(ckpt_dir)
#     tokenizer.save_pretrained(ckpt_dir)

#     # — Commit & Push to Hub —
#     # Copy new checkpoint into the repo root (overwriting previous files)
#     shutil.copytree(ckpt_dir, "training-ucf", dirs_exist_ok=True)
#     repo.git_add(auto_lfs_track=True)
#     repo.git_commit(f"Epoch {epoch+1} — train_loss={avg_train:.4f}, "
#                     f"val_loss={avg_val:.4f}, ROUGE={rouge_scores}")
#     repo.git_push()

In [17]:
import os
import shutil
from subprocess import CalledProcessError
from huggingface_hub import Repository
from torch.amp import autocast
from torch.cuda.amp import GradScaler
import torch
import numpy as np
import nltk
from tqdm.auto import tqdm
import evaluate
from transformers import VisionEncoderDecoderModel, AutoTokenizer

# ─── Setup Rouge, Model, Tokenizer, Repo ─────────────────────────────────────
nltk.download("punkt")
rouge = evaluate.load("rouge")

def postprocess_text(preds, labels):
    preds = ["\n".join(nltk.sent_tokenize(p.strip())) for p in preds]
    labels = ["\n".join(nltk.sent_tokenize(l.strip())) for l in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    processed_labels = [
        [(tok if tok != -100 else tokenizer.pad_token_id) for tok in lab]
        for lab in labels
    ]
    decoded_labels = tokenizer.batch_decode(processed_labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = rouge.compute(predictions=decoded_preds,
                           references=decoded_labels,
                           use_stemmer=True)
    return {k: round(v * 100, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
tokenizer.pad_token = tokenizer.eos_token

# repo = Repository(local_dir="training-ucf", clone_from="username/training-ucf")
scaler = GradScaler()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# (Assume train_dataloader, val_dataloader, optimizer, scheduler, epochs are defined…)

for epoch in range(epochs):
    # — Training —
    model.train()
    train_loss = 0.0
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs} • Training"):
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"]
        if labels.dim() == 1:
            labels = labels.unsqueeze(0)
        labels = labels.to(device)

        decoder_attention_mask = (labels != tokenizer.pad_token_id).to(device)

        optimizer.zero_grad()
        with autocast("cuda"):
            outputs = model(
                pixel_values=pixel_values,
                labels=labels,
                decoder_attention_mask=decoder_attention_mask
            )
            loss = outputs.loss
            train_loss += loss.item()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

    torch.cuda.empty_cache()

    # — Validation & ROUGE —
    model.eval()
    val_loss = 0.0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in val_dataloader:
            pixel_values = batch["pixel_values"].to(device)
            labels = batch["labels"]
            if labels.dim() == 1:
                labels = labels.unsqueeze(0)
            labels = labels.to(device)

            decoder_attention_mask = (labels != tokenizer.pad_token_id).to(device)

            with autocast("cuda"):
                outputs = model(
                    pixel_values=pixel_values,
                    labels=labels,
                    decoder_attention_mask=decoder_attention_mask
                )
                val_loss += outputs.loss.item()

            generated_ids = model.generate(
                pixel_values,
                max_length=labels.shape[-1],
                num_beams=4,
                early_stopping=True
            )
            all_preds.extend(generated_ids.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())

    avg_train = train_loss / len(train_dataloader)
    avg_val   = val_loss   / len(val_dataloader)
    rouge_scores = compute_metrics((all_preds, all_labels))
    print(f"Epoch {epoch+1} — train_loss: {avg_train:.4f}, "
          f"val_loss: {avg_val:.4f}, ROUGE: {rouge_scores}")

    # — Cleanup old checkpoint locally —
    prev_ckpt = f"./training-ucf/checkpoint_{epoch}"
    if epoch > 0 and os.path.isdir(prev_ckpt):
        shutil.rmtree(prev_ckpt)

    # — Save new checkpoint locally —
    ckpt_dir = f"./training-ucf/checkpoint_{epoch+1}"
    model.save_pretrained(ckpt_dir)
    tokenizer.save_pretrained(ckpt_dir)

    # — Copy into repo and push (robust) —
    shutil.copytree(ckpt_dir, "training-ucf", dirs_exist_ok=True)
    repo.git_add(auto_lfs_track=True)

    try:
        repo.git_commit(
            f"Epoch {epoch+1} — train_loss={avg_train:.4f}, "
            f"val_loss={avg_val:.4f}, ROUGE={rouge_scores}"
        )
        repo.git_push()
    except (CalledProcessError, OSError) as e:
        # nothing to commit, or other git error → skip without crashing
        print(f"[Warning] git commit/push skipped: {e}")


/tmp/ipykernel_19/1640549148.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Epoch 1/4 • Training:   0%|          | 0/8397 [00:00<?, ?it/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Epoch 1 — train_loss: 0.1140, val_loss: 0.0859, ROUGE: {'rouge1': 20.7875, 'rouge2': 0.685, 'rougeL': 18.1761, 'rougeLsum': 18.2721}


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 50, 'early_stopping': True, 'num_beams': 4}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


...

To https://huggingface.co/NourFakih/TimeSformer-GPT2-UCF-8400
   34b28eb..74b7692  main -> main



Epoch 4/4 • Training:   0%|          | 0/8397 [00:00<?, ?it/s]

Epoch 4 — train_loss: 0.0745, val_loss: 0.0777, ROUGE: {'rouge1': 22.9414, 'rouge2': 0.4085, 'rougeL': 19.1035, 'rougeLsum': 19.3048}


Upload file checkpoint_4/model.safetensors:   0%|          | 1.00/1.02G [00:00<?, ?B/s]

To https://huggingface.co/NourFakih/TimeSformer-GPT2-UCF-8400
   74b7692..c0924d4  main -> main



## Save on Huggingface (push)

In [19]:
# model_name = "training-ucf"
# hub_repo = f"NourFakih/{model_name}"

# # Save locally
# trainer.save_model(f"./{model_name}")
# tokenizer.save_pretrained(f"./{model_name}")
# feature_extractor.save_pretrained(f"./{model_name}")
# model.save_pretrained(f"./{model_name}")

# # Push to Hugging Face Hub
# tokenizer.push_to_hub(hub_repo)
# feature_extractor.push_to_hub(hub_repo)
# model.push_to_hub(hub_repo)


## Test

In [20]:
import os
import av
import numpy as np
import pandas as pd
import torch
from transformers import AutoImageProcessor, AutoTokenizer, VisionEncoderDecoderModel

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# Load the models
def load_model_and_processor(model_name):
    processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    model = VisionEncoderDecoderModel.from_pretrained(model_name).to(device)
    return processor, tokenizer, model

processor1, tokenizer1, model1 = load_model_and_processor("Neleac/timesformer-gpt2-video-captioning")
processor2, tokenizer2, model2 = load_model_and_processor("/kaggle/working/training-ucf")

# Caption generation function
def generate_caption(video_path, processor, tokenizer, model, num_frames=16):
    try:
        container = av.open(video_path)
        total_frames = container.streams.video[0].frames
        indices = set(np.linspace(0, total_frames, num=num_frames, endpoint=False).astype(np.int64))
        frames = []
        container.seek(0)
        for i, frame in enumerate(container.decode(video=0)):
            if i in indices:
                frames.append(frame.to_ndarray(format="rgb24"))
        pixel_values = processor(frames, return_tensors="pt").pixel_values.to(device)
        tokens = model.generate(pixel_values, min_length=10, max_length=20, num_beams=8)
        return tokenizer.batch_decode(tokens, skip_special_tokens=True)[0]
    except Exception as e:
        print(f"Error processing {video_path}: {e}")
        return "ERROR"



Device: cuda


config.json:   0%|          | 0.00/41.2k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.timesformer.modeling_timesformer.TimesformerModel'> is overwritten by shared encoder config: TimesformerConfig {
  "architectures": [
    "TimesformerForVideoClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "attention_type": "divided_space_time",
  "drop_path_rate": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "abseiling",
    "1": "acting in play",
    "2": "adjusting glasses",
    "3": "air drumming",
    "4": "alligator wrestling",
    "5": "answering questions",
    "6": "applauding",
    "7": "applying cream",
    "8": "archaeological excavation",
    "9": "archery",
    "10": "arguing",
    "11": "arm wrestling",
    "12": "arranging flowers",
    "13": "assembling bicycle",
    "14": "assembling computer",
    "15": "attending conference",
    "16": "auctioning",
    "17": "backflip (human)",
    "18": "baking cookies",
    "19": "bandaging",
    "20": "barb

In [21]:
# Load CSV
csv_path = "/kaggle/input/splitted-clips-ucf/splitted_clips_mapping.csv"
video_folder = "/kaggle/input/splitted-clips-ucf/"

df = pd.read_csv(csv_path)
df['video_path'] = df['video_path'].str.replace(r'^\.\/splitted_clips\/', '', regex=True)
df=df[100:105]
# Process each row
captions_1 = []
captions_2 = []

for idx, row in df.iterrows():
    video_file = os.path.join(video_folder, row['video_path'])
    print(f"[{idx+1}/{len(df)}] Processing {video_file}")

    cap1 = generate_caption(video_file, processor1, tokenizer1, model1)
    cap2 = generate_caption(video_file, processor2, tokenizer2, model2)

    captions_1.append(cap1)
    captions_2.append(cap2)

# Add new columns
df['caption_Neleac'] = captions_1
df['caption_NourFakih'] = captions_2

# Save to new CSV
output_csv_path = "video_captions_augmented.csv"
df.to_csv(output_csv_path, index=False)
print(f"\n✅ Captions saved to {output_csv_path}")

[101/5] Processing /kaggle/input/splitted-clips-ucf/Burglary040_x264_clip8.mp4
[102/5] Processing /kaggle/input/splitted-clips-ucf/Burglary040_x264_clip9.mp4
[103/5] Processing /kaggle/input/splitted-clips-ucf/Burglary040_x264_clip10.mp4
[104/5] Processing /kaggle/input/splitted-clips-ucf/Burglary040_x264_clip11.mp4
[105/5] Processing /kaggle/input/splitted-clips-ucf/Burglary040_x264_clip12.mp4

✅ Captions saved to video_captions_augmented.csv


In [22]:
df

,video_path,caption,caption_Neleac,caption_NourFakih
100,Burglary040_x264_clip8.mp4,The man walked around the sofa and came to a d...,A person is sitting in a living room and they ...,man black walked the and the to door the walk...
101,Burglary040_x264_clip9.mp4,"After a while, the man left the living room an...",A young girl is standing in a living room and ...,man black and woman white to door the and to ...
102,Burglary040_x264_clip10.mp4,"The man came out of the room again,",A young girl is walking on a carpeted floor in...,man black and woman white to door the and to ...
103,Burglary040_x264_clip11.mp4,The man walked up the stairs and walked upstairs.,A person is sliding down a carpeted floor whil...,man the to the of stairs the to the of stairs
104,Burglary040_x264_clip12.mp4,The man came down the stairs and entered the r...,A person is sliding down a carpeted floor whil...,man a and woman to door the and to door
